# Evaluation of Debiasing Methods on Semi-Synthetic Data

This notebook evaluates MF_DR, MF_MRDR_JL, MF_DR_BIAS, and MF_DR_V2 methods on semi-synthetic data.

Metrics evaluated:
- Calibration Error (ECE)
- Balancing Error (BMSE)
- DR Bias
- DR Variance

In [ ]:
# Import necessary libraries
import sys
import os
import pickle
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, List
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path for imports
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))), 'ICLR24-Kernel-Balancing'))

# Import baseline methods
from baselines import MF_DR, MF_MRDR_JL, MF_DR_BIAS, MF_DR_V2

# Set random seeds for reproducibility
np.random.seed(2024)
torch.manual_seed(2024)

print("Imports successful!")

## Data Loading and Preprocessing

In [ ]:
def load_and_preprocess_data(data_path: str = "data/synthetic_data") -> Tuple[np.ndarray, np.ndarray, int, int]:
    """
    Load ground truth data and get dimensions, compute propensity scores.
    
    Returns:
        ground_truth: 1D array of conversion labels
        propensity: 1D array of propensity scores
        num_users: number of users
        num_items: number of items
    """
    # Load ground truth
    with open(data_path, "rb") as f:
        ground_truth = pickle.load(f)
    
    # Load dimensions from predicted_matrix file
    with open("data/predicted_matrix", "rb") as f:
        _ = pickle.load(f)  # predictions
        num_users = pickle.load(f)
        num_items = pickle.load(f)
    
    # Calculate propensity scores as in synthetic.py
    propensity = np.copy(ground_truth)
    p = 0.5
    # Note: The original code has repeated assignments, resulting in:
    # propensity[ground_truth == 1] = p^3 = 0.125
    # propensity[ground_truth == 0] = p^4 = 0.0625
    propensity[np.where(propensity == 1)] = p ** 3
    propensity[np.where(propensity == 0)] = p ** 4
    
    print(f"Loaded ground truth with shape: {ground_truth.shape}")
    print(f"Number of users: {num_users}, Number of items: {num_items}")
    print(f"Ground truth unique values: {np.unique(ground_truth)}")
    print(f"Propensity for positive: {p**3:.4f}, for negative: {p**4:.4f}")
    
    return ground_truth, propensity, num_users, num_items

# Load data
ground_truth, propensity, num_users, num_items = load_and_preprocess_data()

In [ ]:
def convert_to_pairs(ground_truth: np.ndarray, num_users: int, num_items: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Convert 1D ground truth array to (user_id, item_id) pairs with labels.
    
    Returns:
        x_all: array of shape (n_samples, 2) with [user_id, item_id]
        y_all: array of shape (n_samples,) with labels
    """
    # Generate all (user, item) pairs
    x_all = []
    for user in range(num_users):
        for item in range(num_items):
            x_all.append([user, item])
    
    x_all = np.array(x_all)
    y_all = ground_truth.flatten()
    
    return x_all, y_all

# Convert to pairs
x_all, y_all = convert_to_pairs(ground_truth, num_users, num_items)
print(f"Total number of (user, item) pairs: {len(x_all)}")
print(f"Positive rate in ground truth: {y_all.mean():.4f}")

In [ ]:
def generate_biased_observations(x_all: np.ndarray, y_all: np.ndarray, 
                                num_users: int, num_items: int,
                                obs_ratio: float = 0.05,
                                selection_bias: float = 0.7) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Generate biased observed data with selection bias.
    
    Args:
        x_all: all (user, item) pairs
        y_all: all labels
        obs_ratio: ratio of data to observe
        selection_bias: strength of selection bias (higher = more bias towards positive items)
    
    Returns:
        x_obs: observed (user, item) pairs
        y_obs: observed labels
        obs_idx: indices of observed samples
    """
    n_total = len(x_all)
    
    # Create observation probabilities with selection bias
    # Items with y=1 have higher probability of being observed
    base_prob = obs_ratio
    obs_probs = np.ones(n_total) * base_prob
    
    # Increase observation probability for positive items
    positive_idx = np.where(y_all == 1)[0]
    obs_probs[positive_idx] = base_prob * (1 + selection_bias)
    
    # Normalize to ensure expected observation ratio
    obs_probs = obs_probs * (obs_ratio * n_total) / obs_probs.sum()
    obs_probs = np.clip(obs_probs, 0, 1)
    
    # Sample observations
    obs_mask = np.random.binomial(1, obs_probs).astype(bool)
    obs_idx = np.where(obs_mask)[0]
    
    x_obs = x_all[obs_idx]
    y_obs = y_all[obs_idx]
    
    print(f"Generated {len(x_obs)} biased observations ({len(x_obs)/n_total:.2%} of total)")
    print(f"Positive rate in observations: {y_obs.mean():.4f} (vs {y_all.mean():.4f} in ground truth)")
    
    return x_obs, y_obs, obs_idx

# Generate biased observations
x_obs, y_obs, obs_idx = generate_biased_observations(x_all, y_all, num_users, num_items)

In [ ]:
def create_train_test_splits(x_all: np.ndarray, y_all: np.ndarray, 
                           x_obs: np.ndarray, y_obs: np.ndarray,
                           obs_idx: np.ndarray,
                           test_ratio: float = 0.2,
                           y_ips_ratio: float = 0.05) -> Dict:
    """
    Create train/test splits and sample unbiased data (y_ips).
    
    Returns:
        Dictionary with train/test data and y_ips
    """
    # Split observed data into train/test
    x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(
        x_obs, y_obs, obs_idx, test_size=test_ratio, random_state=42, stratify=y_obs
    )
    
    # Sample y_ips (unbiased data) - 5% of all data, excluding test set
    all_idx = np.arange(len(x_all))
    available_idx = np.setdiff1d(all_idx, idx_test)  # Exclude test set
    
    n_ips = int(len(x_all) * y_ips_ratio)
    ips_idx = np.random.choice(available_idx, size=n_ips, replace=False)
    
    x_ips = x_all[ips_idx]
    y_ips = y_all[ips_idx]
    
    # Get true labels for test set (for evaluation)
    y_test_true = y_all[idx_test]
    
    print(f"\nData splits:")
    print(f"Train: {len(x_train)} samples, positive rate: {y_train.mean():.4f}")
    print(f"Test: {len(x_test)} samples, positive rate: {y_test.mean():.4f}")
    print(f"Y_ips (unbiased): {len(x_ips)} samples, positive rate: {y_ips.mean():.4f}")
    
    return {
        'x_train': x_train,
        'y_train': y_train,
        'x_test': x_test,
        'y_test': y_test,
        'y_test_true': y_test_true,
        'x_ips': x_ips,
        'y_ips': y_ips,
        'x_all': x_all,
        'y_all': y_all
    }

# Create train/test splits
data_splits = create_train_test_splits(x_all, y_all, x_obs, y_obs, obs_idx)

## Evaluation Metrics Implementation

In [ ]:
def expected_calibration_error(y_true: np.ndarray, y_pred: np.ndarray, n_bins: int = 10) -> float:
    """
    Calculate Expected Calibration Error (ECE).
    
    ECE measures the difference between predicted probabilities and actual outcomes.
    Lower ECE indicates better calibration.
    """
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
    
    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Find predictions in this bin
        in_bin = (y_pred > bin_lower) & (y_pred <= bin_upper)
        prop_in_bin = in_bin.mean()
        
        if prop_in_bin > 0:
            # Calculate accuracy in this bin
            accuracy_in_bin = y_true[in_bin].mean()
            # Average confidence in this bin
            avg_confidence_in_bin = y_pred[in_bin].mean()
            # Weighted absolute difference
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece

# Test ECE
test_y_true = np.array([0, 0, 1, 1])
test_y_pred = np.array([0.1, 0.2, 0.8, 0.9])
print(f"Test ECE: {expected_calibration_error(test_y_true, test_y_pred):.4f}")

In [ ]:
def balancing_mse(x: np.ndarray, propensity_scores: np.ndarray, 
                  treated_idx: np.ndarray) -> float:
    """
    Calculate Balancing Mean Squared Error (BMSE).
    
    BMSE measures how well the propensity scores balance the covariates.
    Lower BMSE indicates better balancing.
    """
    # Create treatment indicator
    n = len(x)
    treatment = np.zeros(n)
    treatment[treated_idx] = 1
    
    # Calculate inverse propensity weights
    weights = np.zeros(n)
    weights[treatment == 1] = 1 / propensity_scores[treatment == 1]
    weights[treatment == 0] = 1 / (1 - propensity_scores[treatment == 0])
    
    # Normalize weights
    weights[treatment == 1] /= weights[treatment == 1].sum()
    weights[treatment == 0] /= weights[treatment == 0].sum()
    
    # Calculate weighted means for treated and control
    x_treated = x[treatment == 1]
    x_control = x[treatment == 0]
    weights_treated = weights[treatment == 1]
    weights_control = weights[treatment == 0]
    
    # Weighted mean difference
    if len(x_treated) > 0 and len(x_control) > 0:
        weighted_mean_treated = np.average(x_treated, weights=weights_treated, axis=0)
        weighted_mean_control = np.average(x_control, weights=weights_control, axis=0)
        bmse = np.mean((weighted_mean_treated - weighted_mean_control) ** 2)
    else:
        bmse = 0.0
    
    return bmse

print("BMSE metric implemented.")

In [ ]:
def doubly_robust_metrics(y_true: np.ndarray, y_pred: np.ndarray, 
                         y_obs: np.ndarray, propensity_scores: np.ndarray,
                         n_bootstrap: int = 100) -> Tuple[float, float]:
    """
    Calculate DR Bias and DR Variance using bootstrap.
    
    Returns:
        dr_bias: Bias of the doubly robust estimator
        dr_variance: Variance of the doubly robust estimator
    """
    n = len(y_pred)
    dr_estimates = []
    
    # True value (using all ground truth data)
    true_value = y_true.mean()
    
    # Bootstrap to estimate bias and variance
    for _ in range(n_bootstrap):
        # Bootstrap sample
        idx = np.random.choice(n, size=n, replace=True)
        
        # DR estimator components
        y_obs_b = y_obs[idx]
        y_pred_b = y_pred[idx]
        ps_b = propensity_scores[idx]
        
        # Clip propensity scores to avoid division by zero
        ps_b = np.clip(ps_b, 0.01, 0.99)
        
        # DR estimator: E[Y] = E[Y_pred] + E[(Y_obs - Y_pred) / ps]
        dr_estimate = np.mean(y_pred_b) + np.mean((y_obs_b - y_pred_b) / ps_b)
        dr_estimates.append(dr_estimate)
    
    dr_estimates = np.array(dr_estimates)
    
    # Calculate bias and variance
    dr_bias = np.abs(np.mean(dr_estimates) - true_value)
    dr_variance = np.var(dr_estimates)
    
    return dr_bias, dr_variance

print("DR metrics implemented.")

## Training and Evaluation Functions

In [ ]:
def evaluate_model(model, data_splits: Dict, model_name: str) -> Dict[str, float]:
    """
    Train model and evaluate all metrics.
    """
    print(f"\n{'='*50}")
    print(f"Evaluating {model_name}")
    print(f"{'='*50}")
    
    # Extract data
    x_train = data_splits['x_train']
    y_train = data_splits['y_train']
    x_test = data_splits['x_test']
    y_test = data_splits['y_test']
    y_ips = data_splits['y_ips']
    
    # Train model
    print(f"Training {model_name}...")
    
    # Model-specific training parameters
    if model_name == "MF_DR_V2":
        # MF_DR_V2 doesn't use y_ips parameter, it learns propensity internally
        model.fit(x_train, y_train, 
                 num_epoch=100, lr=0.01, verbose=False)
    else:
        model.fit(x_train, y_train, y_ips=y_ips, 
                 num_epoch=100, batch_size=128, lr=0.01, verbose=False)
    
    # Make predictions on test set
    y_pred = model.predict(x_test)
    
    # Apply sigmoid if needed (some models return logits)
    if y_pred.min() < 0 or y_pred.max() > 1:
        y_pred = 1 / (1 + np.exp(-y_pred))
    
    # Calculate ECE
    ece = expected_calibration_error(y_test, y_pred)
    
    # Estimate propensity scores for BMSE calculation
    # Using observed frequency as a simple propensity estimate
    obs_rate = len(x_train) / (num_users * num_items)
    propensity_scores = np.ones(len(x_test)) * obs_rate
    
    # Find which test samples were positive in training
    positive_in_train = []
    for i, (u, it) in enumerate(x_test):
        train_match = (x_train[:, 0] == u) & (x_train[:, 1] == it)
        if train_match.any() and y_train[train_match][0] == 1:
            positive_in_train.append(i)
    
    # Calculate BMSE
    bmse = balancing_mse(x_test, propensity_scores, np.array(positive_in_train))
    
    # Calculate DR Bias and Variance
    dr_bias, dr_variance = doubly_robust_metrics(
        y_test, y_pred, y_test, propensity_scores, n_bootstrap=50
    )
    
    # Store results
    results = {
        'model': model_name,
        'ECE': ece,
        'BMSE': bmse,
        'DR_Bias': dr_bias,
        'DR_Variance': dr_variance
    }
    
    print(f"\nResults for {model_name}:")
    print(f"ECE: {ece:.4f}")
    print(f"BMSE: {bmse:.4f}")
    print(f"DR Bias: {dr_bias:.4f}")
    print(f"DR Variance: {dr_variance:.4f}")
    
    return results

## Train and Evaluate Each Baseline Method

In [ ]:
# Initialize models
embedding_k = 64
batch_size = 128

models = {
    'MF_DR': MF_DR(num_users=num_users, num_items=num_items, embedding_k=embedding_k),
    'MF_MRDR_JL': MF_MRDR_JL(num_users=num_users, num_items=num_items, embedding_k=embedding_k),
    'MF_DR_BIAS': MF_DR_BIAS(num_users=num_users, num_items=num_items, embedding_k=embedding_k),
    'MF_DR_V2': MF_DR_V2(num_users=num_users, num_items=num_items, batch_size=batch_size, embedding_k=embedding_k)
}

# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

for model in models.values():
    model.to(device)

In [ ]:
# Evaluate MF_DR
results_mf_dr = evaluate_model(models['MF_DR'], data_splits, 'MF_DR')

In [ ]:
# Evaluate MF_MRDR_JL
results_mf_mrdr = evaluate_model(models['MF_MRDR_JL'], data_splits, 'MF_MRDR_JL')

In [ ]:
# Evaluate MF_DR_BIAS
results_mf_dr_bias = evaluate_model(models['MF_DR_BIAS'], data_splits, 'MF_DR_BIAS')

In [ ]:
# Evaluate MF_DR_V2
results_mf_dr_v2 = evaluate_model(models['MF_DR_V2'], data_splits, 'MF_DR_V2')

## Results Summary and Visualization

In [ ]:
# Compile all results
all_results = [results_mf_dr, results_mf_mrdr, results_mf_dr_bias, results_mf_dr_v2]
results_df = pd.DataFrame(all_results)

# Display results table
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)
print(results_df.to_string(index=False, float_format='%.4f'))

# Save results
results_df.to_csv('evaluation_results.csv', index=False)
print("\nResults saved to 'evaluation_results.csv'")

In [ ]:
# Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

metrics = ['ECE', 'BMSE', 'DR_Bias', 'DR_Variance']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    values = results_df[metric].values
    models_names = results_df['model'].values
    
    bars = ax.bar(models_names, values, color=colors)
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('Model', fontsize=12)
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.annotate(f'{value:.4f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    # Rotate x-axis labels for better readability
    ax.set_xticklabels(models_names, rotation=45, ha='right')

plt.tight_layout()
plt.savefig('evaluation_metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to 'evaluation_metrics_comparison.png'")

In [ ]:
# Ranking analysis
print("\n" + "="*50)
print("MODEL RANKING BY METRIC")
print("="*50)

for metric in metrics:
    if metric in ['ECE', 'BMSE', 'DR_Bias', 'DR_Variance']:
        # Lower is better for all metrics
        sorted_df = results_df.sort_values(metric)
        print(f"\n{metric} (lower is better):")
        for idx, row in sorted_df.iterrows():
            print(f"  {idx+1}. {row['model']}: {row[metric]:.4f}")